In [103]:
# adapted from Zijin Gu: https://github.com/zijin-gu/scfc-coupling
"""This code takes as input each subject's covariance matrix saved to a .mat file as a variable 'C', and a list
of these matfiles (e.g. from $ 'ls' > subjects.txt).
The output is a .mat file with all subjects' precision FC matrices (C) in the format k*n*n where k = num subjects and n = number of ROIs
as well as a measure of the RMSE for each gamma tested (rmse), and the optimal (opt_gamma)"""

import sys
import scipy.io as sio
import numpy as np
import os

cwd = os.getcwd()
data_dir = '/Users/emilyolafson/GIT/stroke-graph-matching/data'
roi_number = 268

def extract_fc_prec():
    fc_dir =data_dir + '/covariance_may29/'
    subj_all = np.genfromtxt(fc_dir + 'subjects.txt', dtype = 'str')

    fc_corr = np.zeros([subj_all.shape[0],roi_number,roi_number])
    n = 0
    trace=[]
    norm=[]
    for subj in subj_all:
        print(subj)
        fccov = sio.loadmat(fc_dir + str(subj))
        fc_corr[n] = fccov['C'] # need to use covariance
        n = n+1
        
    # regularized precision calculation
    min_rmse = 1e5
    opt_gamma = 0
    allrmse=[]
    
    for gamma in np.linspace(0, 1,50): #for all gamma values
        inverse = []
        reg_inv = []
        
        for i in range(subj_all.shape[0]): #calculate the unregularized inverse and regularized inverse for a given gamma
            np.fill_diagonal(fc_corr[i],1)

            inverse.append(np.linalg.inv(fc_corr[i]))
            reg_inv.append(np.linalg.inv(fc_corr[i] + gamma*np.eye(roi_number)))

        group_prec = np.mean(inverse, axis=0) #calculate the mean unregularized inverse across subjects
        
        diff = []

        # for each subject, calculate the difference between the regularized inverse and group average unregularized inverse.
        for i in range(subj_all.shape[0]):
            diff.append(np.linalg.norm(reg_inv[i][np.triu_indices(roi_number,1)] - group_prec[np.triu_indices(roi_number,1)]))
        rmse = np.mean(diff) #mean difference across subjects
        allrmse.append(rmse)

        if rmse < min_rmse:
            min_rmse = rmse
            opt_gamma = gamma 
            
    fc_prec = np.zeros([subj_all.shape[0],roi_number,roi_number])
    
    opt_gamma=0.714
    #now calculate regularized inversion matrices based on optimal gamma (should be lambda).
    for i in range(subj_all.shape[0]):
        fc_prec[i] = np.linalg.inv(fc_corr[i] + opt_gamma*np.eye(roi_number))

    mdic = {"C": fc_prec, "gamma": opt_gamma, "rmse":allrmse, "sub_sessID": subj_all}
    output_dir =data_dir + '/precision/'

    sio.savemat(output_dir + 'FCprec_concat_allsub.mat', mdic)
    
for sess in range(0,1):
    extract_fc_prec()
    
    

SUB10_S1_shen268_GSR_fc_cov.mat
SUB10_S2_shen268_GSR_fc_cov.mat
SUB10_S3_shen268_GSR_fc_cov.mat
SUB10_S4_shen268_GSR_fc_cov.mat
SUB10_S5_shen268_GSR_fc_cov.mat
SUB11_S1_shen268_GSR_fc_cov.mat
SUB11_S2_shen268_GSR_fc_cov.mat
SUB11_S3_shen268_GSR_fc_cov.mat
SUB11_S4_shen268_GSR_fc_cov.mat
SUB11_S5_shen268_GSR_fc_cov.mat
SUB12_S1_shen268_GSR_fc_cov.mat
SUB12_S2_shen268_GSR_fc_cov.mat
SUB12_S3_shen268_GSR_fc_cov.mat
SUB13_S1_shen268_GSR_fc_cov.mat
SUB13_S2_shen268_GSR_fc_cov.mat
SUB13_S3_shen268_GSR_fc_cov.mat
SUB13_S4_shen268_GSR_fc_cov.mat
SUB13_S5_shen268_GSR_fc_cov.mat
SUB14_S1_shen268_GSR_fc_cov.mat
SUB14_S2_shen268_GSR_fc_cov.mat
SUB14_S3_shen268_GSR_fc_cov.mat
SUB14_S4_shen268_GSR_fc_cov.mat
SUB14_S5_shen268_GSR_fc_cov.mat
SUB15_S1_shen268_GSR_fc_cov.mat
SUB15_S2_shen268_GSR_fc_cov.mat
SUB15_S3_shen268_GSR_fc_cov.mat
SUB15_S4_shen268_GSR_fc_cov.mat
SUB15_S5_shen268_GSR_fc_cov.mat
SUB16_S1_shen268_GSR_fc_cov.mat
SUB16_S2_shen268_GSR_fc_cov.mat
SUB16_S3_shen268_GSR_fc_cov.mat
SUB16_S4

In [2]:
opt_gamma = output_prec['gamma'] 

NameError: name 'output_prec' is not defined

In [101]:
cwd = os.getcwd()
data_dir = '/Users/emilyolafson/GIT/stroke-graph-matching/data'
fc_dir =data_dir + '/controls/precision/'
output_prec = sio.loadmat(fc_dir + 'FCprec_concat_allsub.mat')
rmse = output_prec['rmse'] 
opt_gamma = output_prec['gamma'] 
gridsearch=np.linspace(0,1,50)

#figure(num=None, figsize=(6, 6), dpi=200, facecolor='w', edgecolor='k')

plt.plot(gridsearch,rmse.transpose())
#plt.ylabel('RMSE')
#plt.xlabel('Lambda')
#plt.title('RMSE vs regularization parameter (lambda)')
#plt.text(opt_gamma-0.3, 0.5, 'optimal lambda = ' + str(round(opt_gamma[0][0],3)))
#results_dir = '/Users/emilyolafson/GIT/stroke-graph-matching/apaper/figs/'
#plt.savefig(results_dir + 'optimal_gamma_ctrl.png')

NameError: name 'plt' is not defined

In [102]:
output_prec['gamma']


array([[0.71428571]])